# Class 4 - Decision trees and random forest

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns; sns.set(color_codes=True);
from scipy import stats
from sklearn.model_selection import train_test_split

**Downloading and pre-processing dataset**

We'll use IMDB 5000 Movies dataset in the analysis

In [ ]:
dataset = pd.read_csv("https://raw.githubusercontent.com/KrainskiL/SRD_Z2021/master/Zajecia4/Cwiczenia4_dane.csv")
dataset.head()

Comprehensive data report may be generated using `pandas_profiling` package (see **DatasetReport.html** on repo). If you are interested in the package please check [Pandas-profiling official docs](https://pandas-profiling.github.io/pandas-profiling/docs/)

Code required to generate the report:

```python
import pandas as pd
import pandas_profiling
dataset = pd.read_csv("<filepath>")
report = dataset.profile_report(sort='None', html={'style':{'full_width': True}}, progress_bar=True)
report #to show report in Jupyter
report.to_file(output_file="DatasetReport.html")
```

In [ ]:
#Inspecting columns
dataset.columns

In [ ]:
#Checking numeric columns
dataset.describe(include = [np.number])

In [ ]:
#Checking string columns
## Count ,number of uniques and most frequent value of non-numeric
dataset.describe(include = ['O']) 

In [ ]:
#Dropping columns with many unique values and imbalanced classes
dataset.drop(['color','director_name','actor_2_name','actor_1_name',
             'movie_title','actor_3_name','plot_keywords',
             'movie_imdb_link','language','country','content_rating'],
            axis = 1, inplace = True)

In [ ]:
#Drop duplicates
print(dataset.shape)
dataset.drop_duplicates(inplace = True)
print(dataset.shape)

In [ ]:
#Check null values
dataset.isnull().sum()

In [ ]:
#Dropping missing values
dataset.dropna(inplace=True)
dataset.shape

In [ ]:
#Splitting genres column values and one-hot encoding it
# Getting distinct categories
categories = set(dataset.genres.str.split("|").explode())

# one-hot encode each movie's classification
for cat in categories:
    dataset[cat] = dataset.genres.transform(lambda s: int(cat in s))
dataset.head()

In [ ]:
#Drop genres column
dataset.drop('genres',axis=1,inplace=True)

In [ ]:
stats.zscore(dataset)

In [ ]:
#Removing outliers
dataset=dataset[(np.abs(stats.zscore(dataset)) < 9).all(axis=1)]
dataset.shape

**Splitting data into train and test subsets**

In [ ]:
X = dataset.drop('imdb_score', axis = 1)
y = dataset['imdb_score']
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.15, random_state = 42)

**Short EDA**

In [ ]:
f, axes = plt.subplots(4, 4, figsize =[10,10])
plt.tight_layout(pad=0.4, w_pad=1.0, h_pad=1.0)
for n,col in enumerate(dataset.columns[0:16]):
    sns.regplot(x=col, y="imdb_score", data=dataset, ax=axes[n//4,n%4])

In [ ]:
f, axes = plt.subplots(6, 4, figsize =[10,10])
plt.tight_layout(pad=0.4, w_pad=1.0, h_pad=1.0)
for n,col in enumerate(dataset.columns[16:]):
    sns.barplot(x=col, y="imdb_score", data=dataset, ax=axes[n//4,n%4])
f.delaxes(axes[5,2]);f.delaxes(axes[5,3]);

**Decision Trees**

From: [sklearn docs](https://scikit-learn.org/stable/modules/tree.html#tree-algorithms-id3-c4-5-c5-0-and-cart)

_What are all the various decision tree algorithms and how do they differ from each other? Which one is implemented in scikit-learn?_

**ID3** (Iterative Dichotomiser 3) was developed in 1986 by Ross Quinlan. The algorithm creates a multiway tree, finding for each node (i.e. in a greedy manner) the categorical feature that will yield the largest information gain for categorical targets. Trees are grown to their maximum size and then a pruning step is usually applied to improve the ability of the tree to generalise to unseen data.

**C4.5** is the successor to ID3 and removed the restriction that features must be categorical by dynamically defining a discrete attribute (based on numerical variables) that partitions the continuous attribute value into a discrete set of intervals. C4.5 converts the trained trees (i.e. the output of the ID3 algorithm) into sets of if-then rules. These accuracy of each rule is then evaluated to determine the order in which they should be applied. Pruning is done by removing a rule’s precondition if the accuracy of the rule improves without it.

**C5.0** is Quinlan’s latest version release under a proprietary license. It uses less memory and builds smaller rulesets than C4.5 while being more accurate.

**CART** (Classification and Regression Trees) is very similar to C4.5, but it differs in that it supports numerical target variables (regression) and does not compute rule sets. CART constructs binary trees using the feature and threshold that yield the largest information gain at each node.

**scikit-learn uses an optimised version of the CART algorithm**; however, scikit-learn implementation does not support categorical variables for now.

CART algorithm pick variables and cutoff threshold using:
 1. __for classification__: minimization of node's heterogeneity (Gini index or entropy) 
 2. __for regression__: minimizing error of prediction (e.g. sum of squares of residuals)

In [ ]:
from sklearn import tree

In [ ]:
CART = tree.DecisionTreeRegressor(random_state=42,ccp_alpha=0.0)
CART_model=CART.fit(X_train,y_train)

In [ ]:
CART_model.get_depth()

In [ ]:
CART_model.get_n_leaves()

**Pruning CART tree (cost based)**

⚠️ Requires scikit-learn >=0.22

[Minimal Cost-Complexity Pruning](https://scikit-learn.org/stable/modules/tree.html#minimal-cost-complexity-pruning)

In [ ]:
path = CART.cost_complexity_pruning_path(X_train, y_train)
ccp_alphas, impurities = path.ccp_alphas[::10], path.impurities[::10]
fig, ax = plt.subplots()
ax.plot(ccp_alphas[:-1], impurities[:-1], marker='o', drawstyle="steps-post")
ax.set_xlabel("effective alpha")
ax.set_ylabel("total impurity of leaves")
ax.set_title("Total Impurity vs effective alpha for training set");

In [ ]:
clfs = []
for ccp_alpha in ccp_alphas:
    clf = tree.DecisionTreeRegressor(random_state=42, ccp_alpha=ccp_alpha)
    clf.fit(X_train, y_train)
    clfs.append(clf)
print("Number of nodes in the last tree is: {} with ccp_alpha: {}".format(
      clfs[-1].tree_.node_count, ccp_alphas[-1]))

In [ ]:
def RMSE(model,X,y):
    return np.sqrt(((model.predict(X)-y)**2).mean())

In [ ]:
test_scores = [RMSE(clf,X_test,y_test) for clf in clfs]
train_scores = [RMSE(clf,X_train,y_train) for clf in clfs]

fig, ax = plt.subplots(figsize=[10,10])
ax.set_xlabel("alpha")
ax.set_ylabel("accuracy")
ax.set_title("RMSE vs alpha for training and testing sets")
ax.plot(ccp_alphas, train_scores, marker='o', label="train",
        drawstyle="steps-post")
ax.plot(ccp_alphas, test_scores, marker='o', label="test",
        drawstyle="steps-post")
ax.legend()
plt.show()

In [ ]:
#Complexity (cost) that produce the best regression tree
Best_CART = clfs[np.argmin(test_scores)]
Best_CART.ccp_alpha

In [ ]:
# !conda install python-graphviz -y

In [ ]:
# Exporting printed tree to file
import graphviz
tree.export_graphviz(Best_CART, out_file="cart.dot", 
                     feature_names=X_train.columns,
                     filled=True, rounded=True,  
                     special_characters=True)  

In [ ]:
!dot -Tpng cart.dot -o cart.png

In [ ]:
#RMSE of the best tree
min(test_scores)

In [ ]:
#But we can view this as multiclass classification
confmat = pd.crosstab(Best_CART.predict(X_test).round(),y_test.round())
confmat

In [ ]:
#Accuracy
np.array([confmat.loc[i,i] for i in confmat.index]).sum()/confmat.sum().sum()

### Ensemble tree-based methods

Ensemble learning helps improve final model performance by combining results of underlying models (e.g. random forest is combination of decision trees). This approach allows the production of better predictive performance compared to a single model.

https://scikit-learn.org/stable/modules/ensemble.html#

Two families of ensemble methods are usually distinguished:

- In **averaging methods**, the driving principle is to build several estimators **independently** and then to average their predictions. On average, the combined estimator is usually better than any of the single base estimator because its variance is reduced.

> Example: Random forests

- By contrast, in **boosting methods**, base estimators are built **sequentially** and one tries to reduce the bias of the combined estimator. The motivation is to combine several weak models to produce a powerful ensemble.

> Example: Boosted trees

<img src="https://hpccsystems.com/sites/default/files/inline-images/LearningTrees.PNG" width=400>

[Source](https://hpccsystems.com/blog/learning-trees-guide-to-decision-tree-based-machine-learning)

In [ ]:
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor

**Random forests**

https://scikit-learn.org/stable/modules/ensemble.html#random-forests

[sklearn.ensemble.RandomForestRegressor](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html#sklearn.ensemble.RandomForestRegressor)

We'll look on two parameters:
- n_estimators - number of trees built in ensemble
- max_features - how many features will be included in each tree e.g.
    - 'auto' - all
    - 'sqrt' - random sample of sqrt(n_features)
    - n - random sample of 'n' features

In [ ]:
#Checking number of tress influence on RMSE
rfr = RandomForestRegressor
N = [10,50,100,200,300,400,500]
RMSE_RF= [RMSE(rfr(n,n_jobs=-1).fit(X_train,y_train),X_test,y_test) for n in N]

In [ ]:
plt.plot(N,RMSE_RF,'.-',color='g');
N[np.argmin(RMSE_RF)]

In [ ]:
#Checking number of features influence on RMSE
features = np.linspace(1,X_train.shape[1],10).astype(int)
RMSE_RF_features= [RMSE(rfr(400,max_features=n,n_jobs=-1).fit(X_train,y_train),X_test,y_test) for n in features]

In [ ]:
plt.plot(features,RMSE_RF_features,'.-',color='r');
features[np.argmin(RMSE_RF_features)]

In [ ]:
Best_RF = RandomForestRegressor(400,max_features=25,n_jobs=-1).fit(X_train,y_train)

In [ ]:
# Plot the feature importances of the forest
importances = Best_RF.feature_importances_
std = np.std([tree.feature_importances_ for tree in Best_RF.estimators_],
             axis=0)
indices = np.argsort(importances)[::-1]

num_feat = 6
plt.figure(figsize=[10,5])
plt.title("Feature importances")
plt.bar(range(num_feat)[:num_feat], importances[indices][:num_feat],
       color="r", yerr=std[indices][:num_feat], align="center")
plt.xticks(range(num_feat)[:num_feat], X_train.columns[indices])
plt.xlim([-1, num_feat])
plt.show()

**Gradient Boosted Trees**

https://scikit-learn.org/stable/modules/ensemble.html#gradient-tree-boosting


In [ ]:
#Checking number of tress influence on RMSE
gbr = GradientBoostingRegressor
N = [10,50,100,200,300,400,500,600,700]
RMSE_GBT = [RMSE(gbr(n_estimators=n).fit(X_train,y_train),X_test,y_test) for n in N]

plt.plot(N,RMSE_GBT,'.-',color='y');
N[np.argmin(RMSE_GBT)]

In [ ]:
#From 500 trees RMSE reduction is insignificant
Best_GBT = GradientBoostingRegressor(n_estimators=500).fit(X_train,y_train)

In [ ]:
# Comparison of deviance between training and test set
test_score = np.zeros((500,), dtype=np.float64)

for i, y_pred in enumerate(Best_GBT.staged_predict(X_test)):
    test_score[i] = Best_GBT.loss_(y_test, y_pred)

plt.figure(figsize=(10,5))
plt.title('Deviance')
plt.plot(np.arange(500) + 1, Best_GBT.train_score_, 'b-',
         label='Training Set Deviance')
plt.plot(np.arange(500) + 1, test_score, 'r-',
         label='Test Set Deviance')
plt.legend(loc='upper right')
plt.xlabel('Boosting Iterations')
plt.ylabel('Deviance');

In [ ]:
# Plot feature importance
feature_importance = Best_GBT.feature_importances_
# make importances relative to max importance
feature_importance = 100.0 * (feature_importance / feature_importance.max())
sorted_idx = np.argsort(feature_importance)
pos = np.arange(sorted_idx.shape[0]) + .5
num_feat = 6
plt.figure(figsize=[10,5])
plt.barh(pos[-num_feat:], feature_importance[sorted_idx][-num_feat:], align='center')
plt.yticks(pos[-num_feat:], X_train.columns[sorted_idx][-num_feat:])
plt.xlabel('Relative Importance')
plt.title('Variable Importance')
plt.show()

**Comparing results of Decision Tree, Random Forest and Gradient Boosted Trees**

In [ ]:
models = [Best_CART, Best_RF, Best_GBT]
errors = [RMSE(m, X_test, y_test) for m in models]

In [ ]:
plt.bar(['CART','Random Forest','Gradient Boosted Trees'], errors, color=['red', 'green', 'blue'], alpha=0.75);

In [ ]:
# from sklearn.metrics import mean_squared_error